# Installs



In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
%cd drive/MyDrive/Github/DeepLearning_final

[Errno 2] No such file or directory: 'drive/MyDrive/Github/DeepLearning_final'
/content/drive/MyDrive/Github/DeepLearning_final


In [9]:
!git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 3.37 KiB | 90.00 KiB/s, done.
From https://github.com/Jeremylin0904/DeepLearning_final
   9466efc..8f7e3ac  main       -> origin/main
Updating 9466efc..8f7e3ac
Fast-forward
 Notebook.ipynb | 404 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++----
 1 file changed, 387 insertions(+), 17 deletions(-)


In [10]:
!git switch seb

In [ ]:
!pip install timm
!pip install av
!pip install joblib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 43.1 MB/s eta 0:00:00


In [ ]:
import re
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from urllib import request
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from functools import partial
import gc
import av
import joblib

from timm.models.vision_transformer import PatchEmbed

from DataLoading import *
from SiamMae import *
from LabelPropagation import *
from train import *
from utils import *

# Data loading

In [14]:
# Change it in your branch !
root_path = '/content/drive/MyDrive/DL Project/UCF-101'

In [ ]:
train_data = UCF101FullVideo(root=root_path, output_format="TCHW",transform=transforms)
train_data.__len__()
train_loader = DataLoader(train_data, 32, shuffle=True, collate_fn=custom_collate, pin_memory=True, num_workers = 1)

# Model training

In [ ]:
# Model, optimizer setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sim_mae_vit_small_path16 = sim_mae_vit_small_patch16_dec512d8b
model = sim_mae_vit_small_path16().to(device)
# Change in your branch
folder_logs = '/content/drive/MyDrive/DL Project/Trainings/logs.txt'
folder_model = '/content/drive/MyDrive/DL Project/Trainings'

num_epochs = 100
model = train(model, train_loader, folder_logs, folder_model, num_epochs=num_epochs, lr=1e)

# Label propagation

In [ ]:
# Change in your branch
root = '/content/drive/MyDrive/DL Project/DAVIS'

videos_path = os.path.join(root, 'JPEGImages/480p')
labels_path = os.path.join(root, 'Annotations/480p')

## Model loading

In [ ]:
model_path = '/content/drive/MyDrive/DL Project/training/epoch_20.pt'
model = sim_mae_vit_small_patch16_dec512d8b()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.load_state_dict(torch.load(model_path))

## Inference

In [ ]:
name_video = 'bear'
eval_davis(model, name_video, videos_path, labels_path, 20, dino=True)

In [ ]:
video_num = '00001'
show_segm(videos_path, labels_path, name_video, video_num)

## Evaluation

In [ ]:
print(scores_one_vid(videos_path, name_video))